In [1]:
import os
import sys
import pandas as pd

import evo_helpers as hlp
import evo_constants as CTS

In [ ]:
hlp.welcome()

In [ ]:
#from importlib import reload
#reload(evo_helpers)

opt = hlp.option_menu()

In [ ]:
class Evomining:
    """This class creates an evomining object.
    Its methods permit to access to the evomining
    funtionality.
    """

    def __init__(self, centralDB,evo_genomesDB, nat_prodsDB):
        self.central = centralDB
        self.genomesDB = evo_genomesDB
        self.nat_prods = nat_prodsDB

In [ ]:
## obtener las bases de datos del usuario

#from importlib import reload
#reload(evo_helpers)

## option 1 precharged example:
if opt == 1:
    hlp.make_all_evo_headers(CTS.EXMPL_DB["genomes"],
                                 CTS.EXMPL_DB["rast_ids"])
    hlp.join_evo_headers_files()
    hlp.remove_evo_headers_files()
    
    evo_genomes_path = CTS.EVO_GENOMES_DB
    
    #evomining = EvoMining(cts.EXML_DB["central"],
    #                      cts.EXMPL_DB["genomes"],
    #                      cts.EXMPL_DB["nat_prods"])
    #evomining.run()
## option 2 user databases:
else: 
    user_dbs = hlp.get_dbs()
    hlp.make_all_evo_headers(user_dbs["genomes"], 
                             user_dbs["rast_ids"])
    hlp.join_evo_headers_files()
    hlp.remove_evo_headers_files()
    
    user_dbs["genomes"] = CTS.EVO_GENOMES_DB
    
    evomining = EvoMining(user_dbs["central"],
                         user_dbs["genomes"],
                         user_dbs["nat_prods"])
    #evomining.run()

#evomining.run()

## Obtener los archivos .faa con los encabezados en formato evomining:
### Los nuevos .faa individuales se guardan en data/aux_data/  (se borran una vez utilizados)
### La base de datos con encabezados de evomining se guarda en /data/data_bases/evo_genomes_db/

In [ ]:
#from importlib import reload
#reload(evo_helpers)

genomes_db = "/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/example/genomes/los17"
Rast_ids = "/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/example/genomes/los17Rast.ids"
hlp.make_all_evo_headers(genomes_db, Rast_ids)
hlp.join_evo_headers_files()
#hlp.remove_evo_headers_files()

## Aplicar makeblastdb a las bases de datos que necesitaremos:

In [ ]:
hlp.makeblast_db(CTS.EVO_GENOMES_DB)

In [ ]:
hlp.makeblast_db(CTS.EXMPL_DB["central"])


In [ ]:
hlp.makeblast_db(CTS.EXMPL_DB["nat_prods"])

## Obtener los blastp entre la base de datos central vs la base de datos de genomas:

In [ ]:
blastdb_path = CTS.BLASTDBs_PATH + 'evo_genomes_db_blastdb/'
            
hlp.apply_blastp(CTS.EXMPL_DB["central"], blastdb_path)

In [ ]:
CTS.EVO_GENOMES_DB

## Obtener los blastp entre la base de datos de genomas vs la base de datos central:

In [ ]:
blastdb_path = CTS.BLASTDBs_PATH + 'central_blastdb/'
            
hlp.apply_blastp(CTS.EVO_GENOMES_DB, blastdb_path)

## Obtener los BBH:

### Obtener los mejores elementos únicos del blast de metabolismo central vs genomas: 
### Estos se guardarán en un diccionario de la forma:
### { enzima_i : mejor_copia_en_el_organismo_k }
### donde i recorre todas las enzimas en el metabolismo central, y k recorre todos los genomas.
### - Considerar solo con bitscore >= 100 e E_value < 0.001 

In [ ]:
df1 = pd.read_csv('/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/blastp/central_to_genomes/central_to_genomes.blast', sep = "\t", names = CTS.BLAST_COLS)

In [ ]:
central_enzimes = df1['query'].unique() ##Lista con las enzimas del metabolismo central
genomes = hlp.dict_ids_to_names_genomes(CTS.EXMPL_DB["rast_ids"]) ## Diccio con los genomas de la base de genomas

In [ ]:
central_enzimes

In [ ]:
genomes.keys()

In [ ]:
#definimos el diccionario que será la salida:
unique_in_first_blast = dict()
#inicializamos el diccionario tomando por claves todas las enzimas
# y por el momento asignando por valores listas vacías:
for enzime in central_enzimes:
    unique_in_first_blast[enzime] = []
    
#definimos una lista auxiliar que nos permitirá quedarnos con la mejor
# copia de la enzima presente en cada organismo 

#aux_list = list()
for enzime in central_enzimes:
    for genome_id in genomes.keys():
        aux_list = []
        if genome_id != "6666666.385189":
            for index in range(len(df1)):
                if (aux_list == []):
                    if ((enzime == df1["query"][index])&(genome_id in df1["subject"][index])):
                        aux_list = list(df1.iloc[index])
                    else:
                        continue
                else:
                    if ((enzime == df1["query"][index]) & (genome_id in df1["subject"][index]) & (df1["bitscore"][index] > aux_list[11])):
                        aux_list = list(df1.iloc[index])
            if aux_list != []:  
                unique_in_first_blast[enzime].append(aux_list)
                


In [ ]:
unique_in_first_blast["3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none"]

In [ ]:
total = 0
for enz in central_enzimes:
        total = total + len(unique_in_first_blast[enz])
total

### NOTA: En la base de datos de genomas está de más el genoma con id = 6666666.385189

Unique de Genomes vs EF:

In [ ]:
df2 = pd.read_csv('/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/blastp/genomes_to_central/genomes_to_central.blast', sep = "\t", names = CTS.BLAST_COLS )

In [ ]:
df2.head()

In [ ]:
len(genomes.keys())
    

In [ ]:
#definimos el diccionario que será la salida:
unique_in_second_blast = dict()
#inicializamos el diccionario tomando por claves todas las enzimas
# y por el momento asignando por valores listas vacías:
for genome_id in genomes.keys():
    if genome_id != "6666666.385189":
        unique_in_second_blast[genome_id] = []
    else:
        continue

#definimos una lista auxiliar que nos permitirá quedarnos con la mejor
# copia de la enzima presente en cada organismo 


for genome_id in genomes.keys():
    if genome_id != "6666666.385189":
        for enzime in central_enzimes:
            aux_list = []
            for index in range(len(df2)):
                if (aux_list == []):
                    if ((genome_id in df2["query"][index])&(enzime in df2["subject"][index])):
                        aux_list = list(df2.iloc[index])
                    else:
                        continue
                else:
                    if ((genome_id in df2["query"][index]) & (enzime in df2["subject"][index]) & (df2["bitscore"][index] > aux_list[11])):
                    
                        aux_list = list(df2.iloc[index])
            if aux_list != []:  
                unique_in_second_blast[genome_id].append(aux_list)


### Prueba de bio.genbank package:

In [ ]:
from taxonomy_ranks import TaxonomyRanks

In [ ]:
taxa_name = 'Actinomadura hibisca'

rank_taxon = TaxonomyRanks(taxa_name)

In [ ]:
type(rank_taxon)

In [ ]:
rank_taxon.get_lineage_taxids_and_taxanames()


In [ ]:
ranks = ('user_taxa', 'taxa_searched', 'superkingdom', 'kingdom', 'superphylum', 'phylum', 'subphylum', 'superclass', 'class', 'subclass', 'superorder', 'order', 'suborder', 'superfamily', 'family', 'subfamily', 'genus', 'subgenus', 'species')

In [ ]:
for potential_taxid in rank_taxon.lineages:
     for rank in ranks:
         if rank in ('user_taxa', 'taxa_searched'):
             taxon = rank_taxon.lineages[potential_taxid][rank]
             print(potential_taxid, rank, taxon, sep='\t')
         else:
             taxon, taxid_of_taxon = rank_taxon.lineages[potential_taxid][rank]
             print(potential_taxid, rank, taxon, taxid_of_taxon, sep='\t')


## Probar helpers para BBH:
**<center>1.- best_hit_central_to_orgs:</center>**

In [ ]:
import evo_helpers as hlp
import evo_constants as CTS

In [ ]:
blast_File = '/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/blastp/central_to_genomes/central_to_genomes.blast'
rast_Ids = CTS.EXMPL_DB["rast_ids"]

In [ ]:
#Crea el archivo central_to_orgs_bh.uniq:
hlp.best_hit_central_to_orgs(blast_File)

In [ ]:
hlp.ids_to_names_genomes_file(rast_Ids)

### best_hit_central_to_genomes está bien!

In [ ]:
def make_central_file_info():
    """This function creates a file with the names of all
    the elements in central_db.
    """
    central_fasta_file_path = CTS.EXMPL_DB["central"] + "central.fasta"
    output_path = CTS.BBH_aux_files + "central_enzimes.txt"
    with open(central_fasta_file_path,"r") as fasta_file:
        with open(output_path, "w") as output:
            for line in fasta_file:
                if ">" in line:
                    Line = line.split(">")[1]
                    output.write(Line)
    return

make_central_file_info()

In [ ]:
def ids_to_names_genomes_file(file_rast_ids):
    """ This function returns a file with the genome id's as keys
    and the genome's names as values. The input file_rast_ids is the Rast.ids
    file within the genomes' database.
    """
    
    
    output_path = CTS.BBH_aux_files + "ids_to_orgs_names.txt"
    with open(file_rast_ids, "r") as file:
        with open(output_path, "w") as output:
            for line in file:
                Line = line.split("\t")[1] + "-->" + line.split("\t")[2]
                output.write(Line)
            
    return 
rast_ids_File = CTS.EXMPL_DB["rast_ids"]
ids_to_names_genomes_file(rast_ids_File)

# Por hacer:
- Agregar una parte en la que se creen todos los archivos auxiliares al comienzo de la rutina, para que estén disponibles en las subrutinas.

## Prueba best hit 1 con archivos:

In [ ]:
def best_hit_central_to_orgs(blast_file):
    """ This function will obtain the best hit from central to organisms. Its output 
    is a file with these hits. From the output we will obtain the expanded families.
    """
    
    # Charge the dataframe with the blastp file:
    df = pd.read_csv(blast_file, sep = '\t', names = CTS.BLAST_COLS)
    
    #Obtain the different central enzimes:
    #central_enzimes = df['query'].unique()
    
    #Files with the enzimes and orgs info:
    central_enzimes_file = CTS.BBH_aux_files + 'central_enzimes.txt'
    central_enzimes = []
    with open(central_enzimes_file, "r") as central_file:
        for line in central_file:
            if line:
                central_enzimes.append(line.rstrip("\n"))
    
    ids_to_orgs_names_file = CTS.BBH_aux_files + 'ids_to_orgs_names.txt'
    genomes = dict()
    with open(ids_to_orgs_names_file, "r") as orgs_file:
        for line in orgs_file:
            if line:
                key = line.split("-->")[0]
                value = line.split("-->")[1]
                genomes[key] = value
    
    #Obtain a dictionary of the form {genome_id: "organism_name"}
    #genomes = dict_ids_to_names_genomes(rast_ids)
    
    #Define and initialize the output dictionary:
    unique_central_to_genomes = dict()
    for enzime in central_enzimes:
        unique_central_to_genomes[enzime] = []
        
    # Finding the best hit:
    for enzime in central_enzimes:
        for genome_id in genomes.keys():
            aux_list = []
            if genome_id != "6666666.385189":
                for index in range(len(df)):
                    if (aux_list == []):
                        if ((enzime == df["query"][index])&(genome_id in df["subject"][index])):
                            aux_list = list(df.iloc[index])
                        else:
                            continue
                    else:
                        if ((enzime == df["query"][index]) & (genome_id in df["subject"][index]) & (df["bitscore"][index] > aux_list[11])):
                            aux_list = list(df.iloc[index])
                if aux_list != []:  
                    unique_central_to_genomes[enzime].append(aux_list)
     
    return unique_central_to_genomes

In [ ]:
test = best_hit_central_to_orgs(blast_File)
tot = 0
for enz in test:
    tot += len(test[enz])
tot

In [ ]:
def best_hit_central_to_orgs_2(blast_file):
    """ This function will obtain the best hit from central to organisms. Its output 
    is a file with these hits. From the output we will obtain the expanded families.
    """
    
    # Charge the dataframe with the blastp file:
    df = pd.read_csv(blast_file, sep = '\t', names = CTS.BLAST_COLS)
    
    ### SE NECESITA QUE LOS ARCHIVOS AUXILIARES ESTÉN CREADOS ###
    #Files with the enzimes and orgs info:
    
    # Make a list with the enzimes:
    central_enzimes_file = CTS.BBH_aux_files + 'central_enzimes.txt'
    central_enzimes = []
    with open(central_enzimes_file, "r") as central_file:
        for line in central_file:
            if line:
                central_enzimes.append(line.rstrip("\n"))
    
    
    # Make a dictionary with the ids and names of organisms:
    ids_to_orgs_names_file = CTS.BBH_aux_files + 'ids_to_orgs_names.txt'
    genomes = dict()
    with open(ids_to_orgs_names_file, "r") as orgs_file:
        for line in orgs_file:
            if line:
                key = line.split("-->")[0]
                value = line.split("-->")[1]
                genomes[key] = value
    
    
    
    #Define and initialize the output dictionary:
    unique_central_to_genomes = dict()
    for enzime in central_enzimes:
        unique_central_to_genomes[enzime] = []
        
    # Finding the best hit:
    for enzime in central_enzimes:
        for genome_id in genomes.keys():
            aux_list = []
            
            for index in range(len(df)):
                if (aux_list == []):
                    if ((enzime == df["query"][index])&(genome_id in df["subject"][index])):
                        aux_list = list(df.iloc[index])
                    else:
                        continue
                else:
                    if ((enzime == df["query"][index]) & (genome_id in df["subject"][index]) & (df["bitscore"][index] > aux_list[11])):
                        aux_list = list(df.iloc[index])
            if aux_list != []:  
                unique_central_to_genomes[enzime].append(aux_list)
     
    return unique_central_to_genomes

In [ ]:
best_hit_central_to_orgs_2(blast_File)

In [ ]:
test2 = best_hit_central_to_orgs_2(blast_File)
tot1 = 0
for enz in test2:
    tot1 += len(test2[enz])
tot1

In [ ]:
li = list(df1.iloc[0])
li = [str(el) for el in li]
"\t".join(li)

In [ ]:
def best_hit_central_to_orgs_3(blast_file):
    """ This function will obtain the best hit from central to organisms. Its output 
    is a file with these hits. From the output we will obtain the expanded families.
    """
    
    # Charge the dataframe with the blastp file:
    df = pd.read_csv(blast_file, sep = '\t', names = CTS.BLAST_COLS)
    
    ### SE NECESITA QUE LOS ARCHIVOS AUXILIARES ESTÉN CREADOS ###
    #Files with the enzimes and orgs info:
    
    # Make a list with the enzimes:
    central_enzimes_file = CTS.BBH_aux_files + 'central_enzimes.txt'
    central_enzimes = []
    with open(central_enzimes_file, "r") as central_file:
        for line in central_file:
            if line:
                central_enzimes.append(line.rstrip("\n"))
    
    
    # Make a dictionary with the ids and names of organisms:
    ids_to_orgs_names_file = CTS.BBH_aux_files + 'ids_to_orgs_names.txt'
    genomes = dict()
    with open(ids_to_orgs_names_file, "r") as orgs_file:
        for line in orgs_file:
            if line:
                key = line.split("-->")[0]
                value = line.split("-->")[1]
                genomes[key] = value
    
    
    
    #Define and initialize the output dictionary:
    #unique_central_to_genomes = dict()
    #for enzime in central_enzimes:
    #    unique_central_to_genomes[enzime] = []
     
    # Path to best hits file:
    
    central_to_orgs_bh = CTS.BBH_aux_files + "central_to_orgs_BH.uniq"
    
    
    # Finding the best hit:
    best_hits_file = open(central_to_orgs_bh, "w")
    for enzime in central_enzimes:
        for genome_id in genomes.keys():
            aux_list = []
            
            for index in range(len(df)):
                if (aux_list == []):
                    if ((enzime == df["query"][index])&(genome_id in df["subject"][index])):
                        aux_list = list(df.iloc[index])
                    else:
                        continue
                else:
                    if ((enzime == df["query"][index]) & (genome_id in df["subject"][index]) & (df["bitscore"][index] > aux_list[11])):
                        aux_list = list(df.iloc[index])
            if aux_list != []: 
                line = ["{}\t".format(item) for item in aux_list]
                best_hits_file.writelines(line)
                best_hits_file.write("\n")
                
    best_hits_file.close() 
    #return unique_central_to_genomes
    return

In [ ]:
best_hit_central_to_orgs_3(blast_File)

In [ ]:
cadena = "Hola\ta\ntodos."

In [ ]:
cadena

### Prueba de escribir best hits segundo blast en un archivo:

In [ ]:
def best_hit_orgs_to_central(blast_file):
    """ This function will obtain the best hit from organisms to central, i.e., 
    assigns to each position id of a blast hit the corresponding enzime that best 
    hits in that particular place.
    """
    
    #charge dataframe:
    df = pd.read_csv(blast_file, sep = '\t', names = CTS.BLAST_COLS, index_col = False)
    
    #Obtain dictionary with the copy id position to the line of blast
    
    uniq2 = dict()
    for index in df.index:
        actual_line = df.iloc[index]
        copy_id = actual_line["query"].split("|")[1]  #id of the hit position
        if copy_id in uniq2:
            if actual_line["bitscore"] > uniq2[copy_id][11]:
                uniq2[copy_id] = list(actual_line)
        else:
            if actual_line["bitscore"] > 0:
                uniq2[copy_id] = list(actual_line)
    
    
    # Write the file with the best hits:
    
    orgs_to_central_bh = CTS.BBH_aux_files + "orgs_to_central_BH.uniq"  #Agregar ruta a constantes
    best_hits_file = open(orgs_to_central_bh, "w")
     
    for key in uniq2:
        line = ["{}\t".format(item) for item in uniq2[key]] 
        best_hits_file.writelines(line)
        best_hits_file.write("\n")
    
    
    best_hits_file.close()
    return


In [ ]:
path_second_blast = '/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/blastp/genomes_to_central/genomes_to_central.blast'
hlp.best_hit_orgs_to_central(path_second_blast)

In [ ]:
orgs_to_central_bh = CTS.BBH_aux_files + "orgs_to_central_BH.uniq"
DF1 = pd.read_csv(orgs_to_central_bh, sep = "\t", names = CTS.BLAST_COLS, index_col = False)

In [ ]:
central_to_orgs_bh = CTS.BBH_aux_files + "central_to_orgs_BH.uniq"
DF2 = pd.read_csv(central_to_orgs_bh, sep = "\t", names = CTS.BLAST_COLS, index_col = False)

In [ ]:
DF2.head()

In [ ]:
def bbh_detector(central_to_orgs_bhs, orgs_to_central_bhs):
    """ This function obtains the best bidirectional hits between
    the blast of central to organisms and organisms to central respectively.
    """

    # charge dataframes:
    
    df = pd.read_csv(central_to_orgs_bhs, sep = ' ', names = CTS.BLAST_COLS, index_col = False)
    
    df1 = pd.read_csv(orgs_to_central_bhs, sep = ' ', names = CTS.BLAST_COLS, index_col = False)
    
    # Write the file with the bbh:
    
    BBH_file = CTS.BBH_aux_files + "BBH.bbh"  #Agregar ruta a constantes
    bbh_file = open(BBH_file, "w")
     
    for indx in df1.index:
        copy_id = df1.iloc[indx]["query"].split("|")[1]
        enzime = df1.iloc[indx]["subject"]
        for indxx in df.index:
            if copy_id not in df.iloc[indxx]["subject"]:
                continue
            elif enzime not in df.iloc[indxx]["query"]:
                continue
            else:
                #bbh[copy_id] = enzime
                bbh_file.write(copy_id + " -> ")
                bbh_file.write(enzime +"\t")
                bbh_file.write(df1.iloc[indx]["query"]+"\n")
           
    
    bbh_file.close()
    return

In [ ]:
cent_to_org_bh = CTS.BBH_aux_files + "central_to_orgs_BH.uniq"
org_to_cent_bh = CTS.BBH_aux_files + "orgs_to_central_BH.uniq"
hlp.bbh_detector(cent_to_org_bh, org_to_cent_bh)
#df1 = pd.read_csv(org_to_cent_bh, sep = ' ', names = CTS.BLAST_COLS, index_col =False)

In [ ]:
#df1

In [ ]:
import evo_helpers as hlp
bbh_detector(cent_to_org_bh, org_to_cent_bh)

## Comparar los bbh contra los obtenidos en evomining:


In [ ]:
BBH_mio = list()
BBH_mio_path = "/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/aux_data/bbh_aux_files/BBH.bbh"
bbh_mio = open(BBH_mio_path, 'r')
for line in bbh_mio:
    hit = line.split("\t")[0]
    BBH_mio.append(hit)


bbh_mio.close()
BBH_mio_set = set(BBH_mio)

In [ ]:
BBH_evo = list()
BBH_evo_path = "/home/csar/Proyectos/Posdoc/ALL_curado.fasta_MiBIG_DB.faa_los17/blast/BBH/los17/aux_files/vueltalos17.blast.bbh"
bbh_evo = open(BBH_evo_path, 'r')
for line in bbh_evo:
    hit = line.split("\t")[0]
    BBH_evo.append(hit)


bbh_evo.close()
BBH_evo_set = set(BBH_evo)


In [ ]:
BBH_evo_set == BBH_mio_set

In [ ]:

#bbh_set

In [8]:
expanded_families_files = [x for x in os.listdir(CTS.EXPANDED_FAMS) if x.endswith(".fasta")]

In [9]:
expanded_families_files

['expanded_fam_1.fasta', 'expanded_fam_2.fasta']

In [10]:
blastdb_path = CTS.BLASTDBs_PATH + "nat_prods_blastdb/"
h = blastdb_path + blastdb_path.split("/")[-2]

In [11]:
h

'data/data_bases/blast_dbs/nat_prods_blastdb/nat_prods_blastdb'

In [12]:
blastdb_path = CTS.BLASTDBs_PATH + 'central_blastdb/'
h = blastdb_path + blastdb_path.split("/")[-2]
h

'data/data_bases/blast_dbs/central_blastdb/central_blastdb'

In [13]:
mibig_db = CTS.MIBiG_NAT_PRODS_DB + "nat_prods.fasta"
mibig_file = open(mibig_db, "r")
line = mibig_file.readline()
li = line.strip()
print(li)
mibig_file.close()

>BGC0000001|c1|1-1083|+|AEK75490.1|protein_methyltransferase|AEK75490.1


In [15]:
k = "hola"
"h" in k

True

In [52]:
def std_highlighter(x):
    style_lt = "background-color: white; color: black; font-weight: bold; border: solid; text-align: center; font-size:1.2em;"
    style_gt = "background-color: #C04000; color: black; font-weight: bold; border: solid; text-align:center;"
    gt_std = x > x.mean() + x.std()
    return [style_gt if i else style_lt for i in gt_std]

def std2_highlighter(x):
    style_lt = "background-color: white; color: black; font-weight: bold; border: solid; text-align: center; font-size:1.2em;"
    style_gt = "background-color: #C04000; color: black; font-weight: bold; border: solid; text-align:center;"
    gt_std = x > x.mean() + 2*x.std()
    return [style_gt if i else style_lt for i in gt_std]

def mean_highlighter(x):
    style_lt = "background-color: white; color: black; font-weight: bold; border: solid; text-align: center; font-size:1.2em;"
    style_gt = "background-color: #C04000; color: black; font-weight: bold; border: solid; text-align:center;"
    gt_mean = x > x.mean()
    return [style_gt if i else style_lt for i in gt_mean]
def median_highlighter(x):
    style_lt = "background-color: white; color: black; font-weight: bold; border: solid; text-align: center; font-size:1.2em;"
    style_gt = "background-color: #C04000; color: black; font-weight: bold; border: solid; text-align:center;"
    gt_median = x > x.mean()
    return [style_gt if i else style_lt for i in gt_median]

def mode_highlighter(x):
    import statistics as st
    style_lt = "background-color: white; color: black; font-weight: bold; border: solid; text-align: center; font-size:1.2em;"
    style_gt = "background-color: #C04000; color: black; font-weight: bold; border: solid; text-align:center;"
    gt_mode = x > st.mode(x)
    return [style_gt if i else style_lt for i in gt_mode]

def genomes_highlighter(x):
    style_genome = "background-color: white; color: black; font-weight: bold; border: solid; text-align: center; font-size:1.2em;"
    return [style_genome for i in x]
headers = {
    'selector': 'th.col_heading',
    'props': "background-color: OldLace; color: black; font-weight: bold; border: solid; text-align: center; font-size:1.2em;"
} 
def apply_highlighter2(df_heat,function, style_highlighter, headers_style):
    return df_heat.style.apply(function,subset=["fam_1", "fam_2"]).apply(style_highlighter, subset="orgs_ids").hide(axis="index").set_table_styles([headers_style], overwrite = False)

In [59]:
def make_heat_map(criteria):
    import json
    
    json_file_Expfams = open(CTS.EXP_FAMS_JSON, "r")
    exp_fam_dictio = json.load(json_file_Expfams)
    json_file_Expfams.close()
    
    json_file_org = open(CTS.ORGS_IDS_NMS, "r")
    orgs_ids_dict = json.load(json_file_org)
    orgs_ids_list = list(orgs_ids_dict.keys())
    json_file_org.close()
    
    json_file_fams = open(CTS.FAMILIES, "r")
    fams_dict = json.load(json_file_fams)
    families = list(fams_dict.keys())
    json_file_fams.close()
    
    Dictio_DF = {"orgs_ids": orgs_ids_list}
    for fam in families:
        Dictio_DF[fam] = []
    for org in Dictio_DF["orgs_ids"]:
        for key in exp_fam_dictio:
            if org in key:
                fmly = key.split("|")[0]
                Dictio_DF[fmly].append(len(exp_fam_dictio[key]))
    
    #Dictio_DF
    DF_heat = pd.DataFrame.from_dict(Dictio_DF)
    #for indx in DF_heat.index:
    #    Id = DF_heat["organisms"][indx]
    #    DF_heat.at[indx,"organisms"] = orgs_ids_dict[Id]
    
    
    #DF_heat
    headers_style = {
    'selector': 'th.col_heading',
    'props': "background-color: OldLace; color: black; font-weight: bold; border: solid; text-align: center; font-size:1.2em;"
    }
    if criteria == "mean":
        return apply_highlighter2(DF_heat,mean_highlighter, genomes_highlighter, headers_style)
    elif criteria == "median":
        return apply_highlighter2(DF_heat,median_highlighter, genomes_highlighter, headers_style)
    elif criteria == "mean + std":
        return apply_highlighter2(DF_heat,std_highlighter, genomes_highlighter, headers_style)
    elif criteria == "mean + 2*std":
        return apply_highlighter2(DF_heat,std2_highlighter, genomes_highlighter, headers_style)
    elif criteria == "mode":
        return apply_highlighter2(DF_heat,mode_highlighter, genomes_highlighter, headers_style)
    else: 
        return print(f"{criteria} is not a valid criteria\n")

In [60]:
make_heat_map("mean + std")

orgs_ids,fam_1,fam_2
1156913.7,7,1
169133.3,1,1
290340.24,5,1
367928.21,3,1
442563.11,2,1
446462.15,4,1
452863.24,8,1
649831.6,5,1
6666666.104540,6,1
6666666.111158,9,1
